In [ ]:
#TRAINING PIPELINE
import numpy as np
import time
import tensorflow as tf
import datetime
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from tensorboard.plugins.hparams import api as hp
from keras import backend as K
from tensorflow.keras.utils import Sequence
import numpy as np
import math

#PARAMETERS
UNITS = 512
DROPOUT = 0.1
BATCH = 64
LEN_SEQUENCE = 50
EPOCH = 20

class data_sequence(Sequence):

    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) *
        self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size]

        return batch_x, batch_y

#DIRECTORIES    
DATA_PATH = "../input/harry-potter-all/Harry_Potter_all_modified_v3.txt"
SAVED_MODEL_PATH = "../input/harry-potter-all/Best_weights.hdf5"
CHECKPOINT_PATH = "Best_weights.hdf5"

#OPEN DATA
text = open(DATA_PATH, "r", encoding="utf-8").read().lower()

#DATA PREPARATION
CHARS_IN_TEXT = len(text)
ALL_CHARS = sorted(set(text))

char_to_int = {char:i for i, char in enumerate(ALL_CHARS)}
int_to_char = {v: k for k, v in char_to_int.items()}

X = []
y = []

for i in range(0, CHARS_IN_TEXT-LEN_SEQUENCE, 1):
    sequence_X = text[i:i+LEN_SEQUENCE]
    sequence_y = text[i+LEN_SEQUENCE]
    X.append([char_to_int[c] for c in sequence_X])
    y.append(char_to_int[sequence_y])   

patterns = len(X)

X_data = np.reshape(X, (patterns, LEN_SEQUENCE, 1))
X_data = X_data.astype("float16")       
X_data = X_data/float(len(ALL_CHARS))
y_data = np_utils.to_categorical(y)

#CHECKPOINT INITIALIZATION
checkpoint = ModelCheckpoint(CHECKPOINT_PATH, monitor='loss', verbose=1, save_best_only=True, mode='min')

#STOP TRAINING IF NO IMPROVE
early_stop = tf.keras.callbacks.EarlyStopping("loss", patience=2, restore_best_weights=True)

#LOAD WEIGHTS(from dataset)
sequence = data_sequence(X_data, y_data, BATCH)
model = keras.models.load_model(SAVED_MODEL_PATH)
tensorboard_log = tf.keras.callbacks.TensorBoard(f"./{datetime.datetime.now().strftime('%d-%m-%y %H:%M')} L:{LEN_SEQUENCE}_U:{UNITS}_D:{DROPOUT}_B:{BATCH}", histogram_freq=1)

# START TREINING
while True:
    model.fit(sequence, epochs=EPOCH, callbacks=[checkpoint, tensorboard_log, early_stop])

In [ ]:
#TRAIN FROM SCRATCH
import numpy as np
import time
import tensorflow as tf
import datetime
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.utils import Sequence
import numpy as np
import math

#PARAMETERS
UNITS = 512
DROPOUT = 0.1
BATCH = 64
LEN_SEQUENCE = 50
EPOCH = 20

class data_sequence(Sequence):

    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) *
        self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size]

        return batch_x, batch_y

#DIRECTORIES    
DATA_PATH = "../input/harry-potter-all/Harry_Potter_all_modified_v3.txt"
SAVED_MODEL_PATH = "../input/harry-potter-lstm/Best_weights.hdf5"
CHECKPOINT_PATH = "Best_weights.hdf5"

#OPEN DATA
text = open(DATA_PATH, "r", encoding="utf-8").read().lower()

#DATA PREPARATION
CHARS_IN_TEXT = len(text)
ALL_CHARS = sorted(set(text))

char_to_int = {char:i for i, char in enumerate(ALL_CHARS)}
int_to_char = {v: k for k, v in char_to_int.items()}

X = []
y = []

for i in range(0, CHARS_IN_TEXT-LEN_SEQUENCE, 1):
    sequence_X = text[i:i+LEN_SEQUENCE]
    sequence_y = text[i+LEN_SEQUENCE]
    X.append([char_to_int[c] for c in sequence_X])
    y.append(char_to_int[sequence_y])   

patterns = len(X)

X_data = np.reshape(X, (patterns, LEN_SEQUENCE, 1))
X_data = X_data.astype("float16")       
X_data = X_data/float(len(ALL_CHARS))
y_data = np_utils.to_categorical(y)

#CREATE MODEL
model = Sequential()
model.add(LSTM(UNITS, input_shape=(X_data.shape[1], X_data.shape[2]), return_sequences=True))
model.add(Dropout(DROPOUT))
model.add(LSTM(UNITS))
model.add(Dropout(DROPOUT))
model.add(Dense(y_data.shape[1], activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam")

#CHECKPOINT INITIALIZATION
checkpoint = ModelCheckpoint(CHECKPOINT_PATH, monitor='loss', verbose=1, save_best_only=True, mode='min')

sequence = data_sequence(X_data, y_data, BATCH)
tensorboard_log = tf.keras.callbacks.TensorBoard(f"./{datetime.datetime.now().strftime('%d-%m-%y %H:%M')} L:{LEN_SEQUENCE}_U:{UNITS}_D:{DROPOUT}_B:{BATCH}", histogram_freq=1)
model.fit(sequence, epochs=EPOCH, callbacks=[checkpoint, tensorboard_log])

In [ ]:
#parametr tunning with tensorboard

#PARAMETERS
EPOCHS = 1
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([128]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.2]))
HP_BATCH = hp.HParam('batch', hp.Discrete([2,4,8,16,32,64,128]))
HP_TIME = hp.HParam('time')
HP_SEQUENCE = hp.HParam('Len_sequence')

#HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

#Directory to save logs
log_dir = "./logs/"

with tf.summary.create_file_writer(log_dir).as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_BATCH, HP_TIME, HP_SEQUENCE],
        metrics=[
        hp.Metric(tag="categorical_crossentropy", display_name="Loss")
        ]
    )
        
    
#defining model with changing parameters    
def train_test_model(hparams, directory):
    model = Sequential()
    model.add(LSTM(hparams[HP_NUM_UNITS], input_shape=(X_data.shape[1], X_data.shape[2])))
    model.add(Dropout(hparams[HP_DROPOUT]))
    model.add(Dense(y_data.shape[1], activation="softmax"))
    model.compile(
    optimizer="adam",
    loss='categorical_crossentropy'
    )
    sequence = data_sequence(X_data, y_data, hparams[HP_BATCH])
    start_time = time.time()
    history = model.fit(sequence, epochs=EPOCHS, callbacks=[tf.keras.callbacks.TensorBoard(f"{directory}:{batch_size}", histogram_freq=1)])
    duration = time.time() - start_time
    return history, duration

def run_experiment(directory, hparams):
    with tf.summary.create_file_writer(directory).as_default():
        hist, time = train_test_model(hparams, directory)
        hparams["time"] = (time)/EPOCHS
        hparams["Len_sequence"] = LEN_SEQUENCE
        hp.hparams(hparams)
        for step, loss in enumerate(hist.history["loss"]):
                tf.summary.scalar("categorical_crossentropy", loss, step=step)

session_num = 0

#training loop
for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in HP_DROPOUT.domain.values:
        for batch_size in HP_BATCH.domain.values:
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_BATCH: batch_size
                }
            run_name = "/run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run_experiment(log_dir+run_name, hparams)
            
            session_num += 1

In [ ]:
#compress folder to zip file
import shutil
shutil.make_archive("tensorboard", 'zip', "../")

In [ ]:
#change LR
from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.0001)
model.optimizer.lr

In [ ]:
#LOAD DATA FOR TEXT GENERATION 
import numpy as np
import keras
from keras.utils import np_utils

#DIRECTORIES    
DATA_PATH = "../input/harry-potter-all/Harry_Potter_all_modified_v3.txt"
SAVED_MODEL_PATH = "../input/harry-potter-all/Best_weights.hdf5"

#OPEN DATA
text = open(DATA_PATH, "r", encoding="utf-8").read().lower()

#DATA PREPARATION
CHARS_IN_TEXT = len(text)
ALL_CHARS = sorted(set(text))
LEN_SEQUENCE = 50

char_to_int = {char:i for i, char in enumerate(ALL_CHARS)}
int_to_char = {v: k for k, v in char_to_int.items()}

X = []
y = []

for i in range(0, CHARS_IN_TEXT-LEN_SEQUENCE, 1):
    sequence_X = text[i:i+LEN_SEQUENCE]
    sequence_y = text[i+LEN_SEQUENCE]
    X.append([char_to_int[c] for c in sequence_X])
    y.append(char_to_int[sequence_y])   

patterns = len(X)

X_data = np.reshape(X, (patterns, LEN_SEQUENCE, 1))
X_data = X_data.astype("float16")       
X_data = X_data/float(len(ALL_CHARS))
y_data = np_utils.to_categorical(y)

model = keras.models.load_model(SAVED_MODEL_PATH)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
#GENERATE TEXT
import sys
def temp_index(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds[0], 1)
    return np.argmax(probas)

start = np.random.randint(0, len(X)-1)
pattern = X[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
print("")
temperature = 0.4 #LOWER VALUE-MORE GENERIC/REPETITIVE/PREDICTIVE TEXT : HIGHER VALUE-MORE SUPRISING/UNPREDICTABLE TEXT
for i in range(500):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(ALL_CHARS))
    prediction = model.predict(x, verbose=0)
    index = temp_index(prediction, temperature)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")